# O PROBLEMA

Imagine agora que você foi contratado(a) como Expert em Data Analytics por um grande hospital para entender como foi o comportamento da população na época da pandemia da COVID-19 e quais indicadores seriam importantes para o planejamento, caso haja um novo surto da doença.

Apesar de ser contratado(a) agora, a sua área observou que a utilização do estudo do PNAD-COVID 19 do IBGE seria uma ótima base para termos boas respostas ao problema proposto, pois são dados confiáveis. Porém, não será necessário utilizar todas as perguntas realizadas na pesquisa para enxergar todas as oportunidades ali postas.

É sempre bom ressaltar que há dados triviais que precisam estar no projeto, pois auxiliam muito na análise dos dados:

## PNAD-COVID-19 do IBGE

O Head de Dados pediu para que você entrasse na base de dados do PNAD-COVID-19 do IBGE e organizasse esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:

- a. Utilização de no máximo 20 questionamentos realizados na pesquisa;
- b. Utilizar 3 meses para construção da solução;
- c. Caracterização dos sintomas clínicos da população;
- d. Comportamento da população na época da COVID-19;
- e. Características econômicas da sociedade;

Seu objetivo será trazer uma breve análise dessas informações, como foi a organização do banco, as perguntas selecionadas para a resposta do problema e quais seriam as principais ações que o hospital deverá tomar em caso de um novo surto de COVID-19.

- Dica: Leiam com atenção a base de dados e toda a documentação que o site o PNAD – Covid19 traz, principalmente os dicionários, que ajudam e muito no entendimento da Base de Dados.
- Dica 2: Utilizem o que já foi ensinado e consolidado nas outras fases para apresentar a resolução do projeto.

Lembre-se de que você poderá apresentar o desenvolvimento do seu projeto durante as lives com docentes. Essa é uma boa oportunidade para discutir sobre as dificuldades encontradas e pegar dicas valiosas com especialistas e colegas de turma.

Link para a base: https://www.ibge.gov.br/estatisticas/investigacoes-experimentais/estatisticas-experimentais/27946-divulgacao-semanal-pnadcovid1?t=downloads&utm_source=covid19&utm_medium=hotsite&utm_campaign=covid_19

In [ ]:
import warnings 
warnings.filterwarnings(action = 'ignore')

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from pyspark.sql import SparkSession
import findspark
import os
import pandas as pd
import polars as pl
import plotly.express as px
import plotly.graph_objects as go

### Conectando ao MySQL

In [ ]:
'''def mysql_connection(host, user, passwd, database=None):
    engine = create_engine(f'mysql+pymysql://{user}:{passwd}@{host}/{database}')
    return engine.connect()

mysql = mysql_connection('127.0.0.1', 'root', 'admin', 'pnad_covid')'''

### Criando sessão Spark

In [ ]:
os.environ['SPARK_HOME'] = '/opt/spark/'
findspark.init()

In [ ]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
spark

### Importanto dados tratados da view do arquivo CSV

In [ ]:
df = spark.read.csv('dados/dados_exportados/dados_uteis/2023-10-31_pnad_covid_view.csv', sep=',', inferSchema=True, header=True)
df.show(5, truncate=False)

### Análise exploratória de características clínicas

In [ ]:
df_temp = df.createOrReplaceTempView('df_temp') #criando tabela temporária na sessão Spark SQL

#### Qual é a porcetagem de sintomáticos que efetuou medida de isolamento social?

In [ ]:
df_testes_validos_zona = spark.sql(
    '''
        SELECT situacao_domicilio, count(resultado_teste) AS qtd_testes_validos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' OR resultado_teste = 'Negativo' 
        GROUP BY situacao_domicilio
    '''
).toPandas()

df_positivos_zona = spark.sql(
    '''
        SELECT situacao_domicilio, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY situacao_domicilio
    '''
).toPandas()


df_infectados_zona = pd.merge(df_testes_validos_zona, df_positivos_zona, on='situacao_domicilio')
df_infectados_zona['percentual_infectados'] = ((df_infectados_zona.qtd_infectados / df_infectados_zona.qtd_testes_validos) * 100).round(decimals=2)

fig = px.pie(
    data_frame=df_infectados_zona, 
    values = 'percentual_infectados', 
    names = 'situacao_domicilio',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={'percentual_infectados':'Percentual de infectados entre pacientes testados na zona de domicílio',
            'situacao_domicilio':'Zona'
    }
)

fig.update_layout(
    title='<b>Porcentagem de infectados por zona de domicílio</b>',
    legend_title='Legenda',
    width=800, 
    height=600
)

fig.show()